### Imports

In [1]:
import os
import statistics
import numpy as np
from PIL import Image
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import adam_v2
from keras.utils.vis_utils import plot_model

### Pre-processing

In [2]:
IMAGE_SIZE = 128
IMAGE_CHANNELS = 3
PAINTINGS_DIR = '/kaggle/input/dataset/'
real_paintings = []

for filename in os.listdir(PAINTINGS_DIR):
    path = os.path.join(PAINTINGS_DIR, filename)
    image = Image.open(path).resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)

    real_paintings.append(np.asarray(image))

real_paintings = np.reshape(
    real_paintings, (-1, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
real_paintings = real_paintings / 127.5 - 1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  


### Config

In [3]:
NOISE_SIZE = 100
EPOCHS = 500
BATCH_SIZE = 32
DROPOUT_RATE = 0.3

### Discriminator

In [4]:
def build_discriminator(image_shape):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=3, strides=2,
    input_shape=image_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(DROPOUT_RATE))
    
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(DROPOUT_RATE))
    
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(DROPOUT_RATE))
    
    model.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(DROPOUT_RATE))
    
    model.add(Conv2D(512, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(DROPOUT_RATE))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    input_image = Input(shape=image_shape)
    validity = model(input_image)
    
    plot_model(model, to_file='/kaggle/working/discriminator_model_plot.png', show_shapes=True, show_layer_names=True)
   
    return Model(input_image, validity)

### Generator

In [5]:
def build_generator(noise_size, channels):
    model = Sequential()
    
    model.add(Dense(4 * 4 * 256, activation='relu',       input_dim=noise_size))
    model.add(Reshape((4, 4, 256)))
    
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    
    model.add(Conv2D(channels, kernel_size=3, padding='same'))
    model.add(Activation('tanh'))
    input = Input(shape=(noise_size,))
    generated_image = model(input)
    plot_model(model, to_file='/kaggle/working/generator_model_plot.png', show_shapes=True, show_layer_names=True)
    return Model(input, generated_image)


### Output Saving

In [6]:
def save_images(cnt, image, operation="train", tag="fake", path="training-output"): 
    out = None
    if tag == "fake":
        image = generator.predict(image)    
        out = image
    
    image = 0.5 * image + 0.5 
    image = (image[0] * 256).astype(np.uint8) # Save only first image in batch
        
    output_path = f'/kaggle/working/{path}'

    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    filename = os.path.join(output_path, f"{operation}-{cnt}-{tag}.png")
    im = Image.fromarray(image)
    im.save(filename)
    
    return out

### Train & Generate 

In [7]:
image_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)
optimizer = adam_v2.Adam(1.5e-4, 0.5)
discriminator = build_discriminator(image_shape)
discriminator.compile(loss='binary_crossentropy',
optimizer=optimizer, metrics=['accuracy'])
generator = build_generator(NOISE_SIZE, IMAGE_CHANNELS)
random_input = Input(shape=(NOISE_SIZE,))
generated_image = generator(random_input)
discriminator.trainable = False
validity = discriminator(generated_image)
combined = Model(random_input, validity)
combined.compile(loss='binary_crossentropy',
optimizer=optimizer, metrics=['accuracy'])
y_real = np.ones((BATCH_SIZE, 1))
y_fake = np.zeros((BATCH_SIZE, 1))
fixed_noise = np.random.normal(0, 1, (1, NOISE_SIZE))
cnt = 1
for epoch in range(EPOCHS):
    idx = np.random.randint(0, real_paintings.shape[0], BATCH_SIZE)
    x_real = real_paintings[idx]

    noise= np.random.normal(0, 1, (BATCH_SIZE, NOISE_SIZE))
    x_fake = generator.predict(noise)

    discriminator_metric_real = discriminator.train_on_batch(x_real, y_real)
    discriminator_metric_generated = discriminator.train_on_batch(
    x_fake, y_fake)

    discriminator_metric = 0.5 * np.add(discriminator_metric_real, discriminator_metric_generated)
    generator_metric = combined.train_on_batch(noise, y_real)
    
    save_images(cnt, fixed_noise)
    cnt += 1

    print(f'{epoch} epoch, Discriminator accuracy: {100*  discriminator_metric[1]}, Generator accuracy: {100 * generator_metric[1]}')

2023-01-22 15:15:37.414964: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-22 15:15:37.557043: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-22 15:15:37.557920: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-22 15:15:37.559524: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

0 epoch, Discriminator accuracy: 10.9375, Generator accuracy: 100.0
1 epoch, Discriminator accuracy: 57.8125, Generator accuracy: 0.0
2 epoch, Discriminator accuracy: 54.6875, Generator accuracy: 75.0
3 epoch, Discriminator accuracy: 50.0, Generator accuracy: 96.875
4 epoch, Discriminator accuracy: 90.625, Generator accuracy: 100.0
5 epoch, Discriminator accuracy: 79.6875, Generator accuracy: 96.875
6 epoch, Discriminator accuracy: 92.1875, Generator accuracy: 100.0
7 epoch, Discriminator accuracy: 90.625, Generator accuracy: 100.0
8 epoch, Discriminator accuracy: 95.3125, Generator accuracy: 100.0
9 epoch, Discriminator accuracy: 81.25, Generator accuracy: 53.125
10 epoch, Discriminator accuracy: 56.25, Generator accuracy: 28.125
11 epoch, Discriminator accuracy: 39.0625, Generator accuracy: 0.0
12 epoch, Discriminator accuracy: 9.375, Generator accuracy: 0.0
13 epoch, Discriminator accuracy: 31.25, Generator accuracy: 28.125
14 epoch, Discriminator accuracy: 79.6875, Generator accura

### Predict

In [10]:
def sliced_wasserstein(A, B, dir_repeats=128, dirs_per_repeat=4):
    
    A = np.reshape(A, (A.shape[0], -1))
    B = np.reshape(B, (B.shape[0], -1))
    
    assert A.ndim == 2 and A.shape == B.shape                           # (neighborhood, descriptor_component)
    results = []
    for repeat in range(dir_repeats):
        dirs = np.random.randn(A.shape[1], dirs_per_repeat)             # (descriptor_component, direction)
        dirs /= np.sqrt(np.sum(np.square(dirs), axis=0, keepdims=True)) # normalize descriptor components for each direction
        dirs = dirs.astype(np.float32)
        projA = np.matmul(A, dirs)                                      # (neighborhood, direction)
        projB = np.matmul(B, dirs)
        projA = np.sort(projA, axis=0)                                  # sort neighborhood projections for each direction
        projB = np.sort(projB, axis=0)
        dists = np.abs(projA - projB)                                   # pointwise wasserstein distances
        results.append(np.mean(dists))                                  # average over neighborhoods and directions
    return np.mean(results)

SWD = []

# Noise to Paintings
for i in range(len(real_paintings)):
    ix = np.random.randint(0, real_paintings.shape[0], BATCH_SIZE)
    real = real_paintings[ix]
    noise= np.random.normal(0, 1, (BATCH_SIZE, NOISE_SIZE)) # Noise is 32 samples of 128 pixels
                                   
    fake = save_images(i, noise, operation="test", tag="fake", path="test-output")
    save_images(i, real, operation="test", tag="real", path="test-output")
    
    SWD.append(sliced_wasserstein(real, fake))
    
    # Stop Early if needed
    print(i)
    if i == 15:
        break    
        

print(f"Mean SWD: {statistics.mean(SWD)}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Mean SWD: 0.771081699674765


### Download results

In [9]:
os.chdir(r'/kaggle/working')

!tar -czf Out.tar.gz ./

from IPython.display import FileLink

FileLink(r'Out.tar.gz')

tar: .: file changed as we read it


/kaggle/working/Out.tar.gz